In [4]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import csv
import xlrd

In [56]:
reader = csv.reader(open('emoji_scrapped_data.csv'))
emoji_dict = {}
for row in reader:
    key=row[1]
    if key in emoji_dict:
        pass
    emoji_dict[key] = row[0]
#print(emoji_dict)

In [57]:
import emoji
for i in emoji_dict.keys():
    print(i, end=" ")
    print(emoji.emojize(emoji_dict[i], language='alias'))

&#129351; 🥇
&#129352; 🥈
&#129353; 🥉
&#127374; 🆎
&#127975; 🏧
&#127344;&#65039; 🅰️
&#127344; 🅰️
&#127462;&#127467; 🇦🇫
&#127462;&#127473; 🇦🇱
&#127465;&#127487; 🇩🇿
&#127462;&#127480; 🇦🇸
&#127462;&#127465; 🇦🇩
&#127462;&#127476; 🇦🇴
&#127462;&#127470; 🇦🇮
&#127462;&#127478; 🇦🇶
&#127462;&#127468; 🇦🇬
&#9810; ♒
&#127462;&#127479; 🇦🇷
&#9800; ♈
&#127462;&#127474; 🇦🇲
&#127462;&#127484; 🇦🇼
&#127462;&#127464; 🇦🇨
&#127462;&#127482; 🇦🇺
&#127462;&#127481; 🇦🇹
&#127462;&#127487; 🇦🇿
&#128281; 🔙
&#127345;&#65039; 🅱️
&#127345; 🅱️
&#127463;&#127480; 🇧🇸
&#127463;&#127469; 🇧🇭
&#127463;&#127465; 🇧🇩
&#127463;&#127463; 🇧🇧
&#127463;&#127486; 🇧🇾
&#127463;&#127466; 🇧🇪
&#127463;&#127487; 🇧🇿
&#127463;&#127471; 🇧🇯
&#127463;&#127474; 🇧🇲
&#127463;&#127481; 🇧🇹
&#127463;&#127476; 🇧🇴
&#127463;&#127462; 🇧🇦
&#127463;&#127484; 🇧🇼
&#127463;&#127483; 🇧🇻
&#127463;&#127479; 🇧🇷
&#127470;&#127476; 🇮🇴
&#127483;&#127468; 🇻🇬
&#127463;&#127475; 🇧🇳
&#127463;&#127468; 🇧🇬
&#127463;&#127467; 🇧🇫
&#127463;&#127470; 🇧🇮
&#127377; 🆑
&#127378; 🆒
&#

In [2]:
import requests

In [3]:
url = "http://127.0.0.1:5000/movies"

r = requests.get(url)

r

<Response [500]>

In [5]:
responsejson = r.json()

In [6]:
with open('response.csv', 'w', newline = '') as csvfile:
    fieldnames = ["_id", "movie_name", "emoji_array"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in responsejson:
        writer.writerow(row)

In [5]:
data = pd.read_csv('response.csv')
data.head()

,_id,movie_name,emoji_array
0,624bc34a8df43a0264f76532,Boss Fight,&#128104; &#129324;
1,624bce528df43a0264f76533,Holidays,&#127876;
2,624bce7e8df43a0264f76534,House on Haunted Hill,&#127968; &#128123; &#127956;&#65039;
3,624bcf648df43a0264f76535,Dracula 2000,&#129499;&#8205;&#9794;&#65039;
4,624bcf858df43a0264f76536,Arctic Dogs,&#129398; &#128021;


In [8]:
data = data.drop(["_id"], axis=1)

KeyError: "['_id'] not found in axis"

In [10]:
data.head()

,movie_name,emoji_array
0,Boss Fight,&#128104; &#129324;
1,Holidays,&#127876;
2,House on Haunted Hill,&#127968; &#128123; &#127956;&#65039;
3,Dracula 2000,&#129499;&#8205;&#9794;&#65039;
4,Arctic Dogs,&#129398; &#128021;


In [11]:
#lowercase all movie_name letters
data.movie_name=data.movie_name.apply(lambda x: x.lower())

#remove quotes
data.movie_name=data.movie_name.apply(lambda x: re.sub("'", '', x))

#remove special characters
exclude = set(string.punctuation)
data.movie_name=data.movie_name.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

#remove digits
remove_digits = str.maketrans('', '', digits)
data.movie_name=data.movie_name.apply(lambda x: x.translate(remove_digits))

#remove extra spaces
data.movie_name=data.movie_name.apply(lambda x: x.strip())
data.emoji_array=data.emoji_array.apply(lambda x: x.strip())
data.movie_name=data.movie_name.apply(lambda x: re.sub(" +", " ", x))
data.emoji_array=data.emoji_array.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
data.emoji_array = data.emoji_array.apply(lambda x : 'START_ '+ x + ' _END')

In [12]:
data.head()

,movie_name,emoji_array
0,boss fight,START_ &#128104; &#129324; _END
1,holidays,START_ &#127876; _END
2,house on haunted hill,START_ &#127968; &#128123; &#127956;&#65039; _END
3,dracula,START_ &#129499;&#8205;&#9794;&#65039; _END
4,arctic dogs,START_ &#129398; &#128021; _END


In [13]:
# Vocabulary of English
all_movie_name_words=set()
for movie_name in data.movie_name:
    for word in movie_name.split():
        if word not in all_movie_name_words:
            all_movie_name_words.add(word)

# Vocabulary of emoji html encoding 
all_emoji_encodings=set()
for emo in data.emoji_array:
    for word in emo.split():
        if word not in all_emoji_encodings:
            all_emoji_encodings.add(word)

In [33]:
# Max Length of source sequence
lenght_list=[]
for d in data.movie_name:
    lenght_list.append(len(d.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

9

In [35]:
# Max Length of target sequence
lenght_list=[]
for e in data.emoji_array:
    lenght_list.append(len(e.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

5

In [36]:
input_words = sorted(list(all_movie_name_words))
target_words = sorted(list(all_emoji_encodings))
num_encoder_tokens = len(all_movie_name_words)
num_decoder_tokens = len(all_emoji_encodings)
num_encoder_tokens, num_decoder_tokens
num_decoder_tokens += 1
num_decoder_tokens

60

In [37]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [38]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [39]:
data.head()

,movie_name,emoji_array
0,boss fight,START_ &#128104; &#129324; _END
1,holidays,START_ &#127876; _END
2,house on haunted hill,START_ &#127968; &#128123; &#127956;&#65039; _END
3,dracula,START_ &#129499;&#8205;&#9794;&#65039; _END
4,arctic dogs,START_ &#129398; &#128021; _END


In [40]:
X, y = data.movie_name, data.emoji_array
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((27,), (3,))

In [41]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [42]:
def generate_batch(X = X_train, y = y_train, batch_size = 32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [43]:
latent_dim = 50

In [44]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [45]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

In [47]:
model.save_weights('nmt_weights.h5')

In [48]:
model.load_weights('nmt_weights.h5')

In [49]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [50]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [94]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [95]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Emoji Translation:', ''.join([emoji.emojize(emoji_dict[s]) for s in y_train[k:k+1].values[0][6:-4].split()]))
print('Predicted Emoji Translation:', ''.join([emoji.emojize(emoji_dict[s]) for s in decoded_sentence.split()]))

Input English sentence: the bunker game
Actual Emoji Translation: ☢️🎲
Predicted Emoji Translation: 👩‍🦰🤬👾💪
